In [73]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 6.0)

from geopy.geocoders import Nominatim
from geopy.distance import vincenty
import numpy as np
import math
import seaborn as sns
from operator import itemgetter

pd.set_option('display.float_format', lambda x: '%.5f' % x) # pandas
np.set_printoptions(precision=5, suppress=True) # numpy

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# seaborn plotting style
sns.set(style='ticks', context='poster')

## Get CA weather station lat and long

In [27]:
weather_stations = pd.read_csv('./NSRDB_StationsMeta.csv')
weather_stations.head()

,USAF,CLASS,SOLAR,STATION,ST,NSRDB_LAT (dd),NSRDB_LON(dd),NSRDB_ELEV (m),TIME ZONE,ISH_LAT (dd),ISH_LON(dd),ISH_ELEV (m),20070316
0,690140,3,0,EL TORO MCAS,CA,33.66700,-117.73300,116,-8,33.66700,-117.73300,116,nan
1,690150,2,0,TWENTYNINE PALMS,CA,34.30000,-116.16700,626,-8,34.30000,-116.16700,626,nan
2,690160,3,0,TUSTIN MCAF,CA,33.70000,-117.83300,18,-8,33.70000,-117.83300,18,nan
3,690190,2,0,ABILENE DYESS AFB,TX,32.43300,-99.85000,545,-6,32.43300,-99.85000,545,nan
4,690230,2,0,WHIDBEY ISLAND NAS,WA,48.35000,-122.66700,10,-8,48.35000,-122.66700,10,nan


In [84]:
CA_weather_stations = weather_stations[weather_stations['ST'] == 'CA']
CA_weather_stations.head()

,USAF,CLASS,SOLAR,STATION,ST,NSRDB_LAT (dd),NSRDB_LON(dd),NSRDB_ELEV (m),TIME ZONE,ISH_LAT (dd),ISH_LON(dd),ISH_ELEV (m),20070316
0,690140,3,0,EL TORO MCAS,CA,33.66700,-117.73300,116,-8,33.66700,-117.73300,116,nan
1,690150,2,0,TWENTYNINE PALMS,CA,34.30000,-116.16700,626,-8,34.30000,-116.16700,626,nan
2,690160,3,0,TUSTIN MCAF,CA,33.70000,-117.83300,18,-8,33.70000,-117.83300,18,nan
363,722810,3,0,NAF,CA,32.81700,-115.68300,-13,-8,32.81700,-115.68300,-13,nan
364,722860,2,0,MARCH AFB,CA,33.90000,-117.25000,462,-8,33.90000,-117.25000,462,nan


In [85]:
CA_weather_stations = CA_weather_stations.loc[:, ['USAF', 'STATION', 'CLASS', 'ISH_LAT (dd)', 'ISH_LON(dd)']]

In [86]:
CA_weather_stations.rename({'ISH_LAT (dd)':'latitude', 'ISH_LON(dd)':'longitude'}, axis=1, inplace = True)
CA_weather_stations.head()

,USAF,STATION,CLASS,latitude,longitude
0,690140,EL TORO MCAS,3,33.66700,-117.73300
1,690150,TWENTYNINE PALMS,2,34.30000,-116.16700
2,690160,TUSTIN MCAF,3,33.70000,-117.83300
363,722810,NAF,3,32.81700,-115.68300
364,722860,MARCH AFB,2,33.90000,-117.25000


## Create a USAF : station_name dictionary

In [87]:
st_name = dict(zip(CA_weather_stations.USAF, CA_weather_stations.STATION))
st_name

{690140: 'EL TORO MCAS',
 690150: 'TWENTYNINE PALMS',
 690160: 'TUSTIN MCAF',
 722810: 'NAF',
 722860: 'MARCH AFB',
 722865: 'ONTARIO',
 722866: 'NORTON AFB/SAN BERN',
 722867: 'BEAUMONT 1 E',
 722868: 'PALM SPRINGS INTL',
 722869: 'RIVERSIDE MUNI',
 722874: 'LOS ANGELES DOWNTOWN USC CAMP',
 722880: 'BURBANK-GLENDALE-PASSADENA AP',
 722885: 'SANTA MONICA MUNI',
 722886: 'VAN NUYS AIRPORT',
 722895: 'LOMPOC (AWOS)',
 722897: 'SAN LUIS CO RGNL',
 722899: 'CHINO AIRPORT',
 722900: 'SAN DIEGO LINDBERGH FIELD',
 722903: 'SAN DIEGO/MONTGOMER',
 722904: 'CHULA VISTA BROWN FIELD NAAS',
 722906: 'SAN DIEGO NORTH ISLAND NAS',
 722909: 'IMPERIAL BEACH REAM FIELD NAS',
 722910: 'SAN NICHOLAS ISLAND',
 722920: 'SANTA CATALINA CATALINA ARPT',
 722925: 'SAN CLEMENTE IS NAAS',
 722926: 'CAMP PENDLETON MCAS',
 722927: 'CARLSBAD/PALOMAR',
 722930: 'SAN DIEGO MIRAMAR NAS',
 722934: 'OCEANSIDE MUNICIPAL',
 722950: 'LOS ANGELES INTL ARPT',
 722953: 'MOJAVE',
 722955: 'TORRANCE MUNICIPAL ARPT',
 722956: 'JA

## Create a USAF: (Lat, long) dictionary

In [88]:
st_CA = CA_weather_stations.loc[:, ['USAF', 'latitude', 'longitude']]

In [89]:
st_CA.index = st_CA.USAF

In [90]:
st_CA.drop(columns = 'USAF', inplace =True)

In [91]:
st_dict = st_CA.to_dict('index')

In [100]:
st_dict

{690140: {'latitude': 33.667, 'longitude': -117.73299999999999},
 690150: {'latitude': 34.3, 'longitude': -116.167},
 690160: {'latitude': 33.7, 'longitude': -117.833},
 722810: {'latitude': 32.817, 'longitude': -115.68299999999999},
 722860: {'latitude': 33.9, 'longitude': -117.25},
 722865: {'latitude': 34.067, 'longitude': -117.65},
 722866: {'latitude': 34.1, 'longitude': -117.23299999999999},
 722867: {'latitude': 33.933, 'longitude': -116.96700000000001},
 722868: {'latitude': 33.833, 'longitude': -116.5},
 722869: {'latitude': 33.95, 'longitude': -117.45},
 722874: {'latitude': 34.05, 'longitude': -118.23299999999999},
 722880: {'latitude': 34.2, 'longitude': -118.35},
 722885: {'latitude': 34.016999999999996, 'longitude': -118.45},
 722886: {'latitude': 34.217, 'longitude': -118.48299999999999},
 722895: {'latitude': 34.667, 'longitude': -120.46700000000001},
 722897: {'latitude': 35.233000000000004, 'longitude': -120.633},
 722899: {'latitude': 33.967, 'longitude': -117.633},


## Use geopy to get lat, long from an address

In [111]:
addr = 'University of California, Berkeley, CA 94720'
geolocator = Nominatim()

In [112]:
location = geolocator.geocode(addr)
location

Location((37.87631055, -122.238859269, 0.0))

## Caculate distance between this location and all weather station, find the nearest station 

In [113]:
distances = {}
for st in st_dict.items():
    distances[st[0]] = vincenty((location.latitude, location.longitude), (st[1]['latitude'], st[1]['longitude'])).miles

In [114]:
cloest = sorted(distances.items(), key = itemgetter(1))[0][0]

In [115]:
st_name[cloest]

'ALAMEDA NAS'